In [28]:
## library
import random
import numpy as np
import torch
from torch.autograd.variable import Variable
from torch import nn, optim
import argparse
import matplotlib as plt 
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import sys
import visdom
import numpy as np
from keras.utils.vis_utils import plot_model
import time
from tqdm import tqdm
import tensorflow as tf
from torchsummary import summary
from torch.utils.data import Subset




##Classifier pytorch    

[WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée
[WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée


In [27]:
noise_dim=100
batch_size=10
num_img_channel=1
img_size=28
sample_interval=64          



[WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée
[WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée
[WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée
[WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée
[WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée
[WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée
[WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée
[WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée
[WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a expressément refusée
[WinError 10061] Aucune connexion n’a pu être établie car l’ordinateur cible l’a e

In [58]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier,self).__init__()

        
        
        self.conv_layer1=nn.Sequential(
            nn.Conv2d(num_img_channel,16,kernel_size=[6,6],stride=1,padding=1,bias =False,dilation=1),


            nn.LeakyReLU(0.18,inplace=True),
        )

        self.conv_layer2=nn.Sequential(
            nn.Conv2d(16,32,kernel_size=[6,6],stride=1,padding=0,bias =False,dilation=1,),

            nn.LeakyReLU(0.18,inplace=True),
        )
        


        self.fc_block1= nn.Sequential(
            
            nn.Linear(32*20*20,512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.18,inplace=True),
            nn.Linear(512,10)
        )



    def forward(self,img):
        layer1_out=self.conv_layer1(img)
        layer2_out=self.conv_layer2(layer1_out)
        conv_out=layer2_out.view(img.size(0),32*20*20)
        l2_value = self.fc_block1(conv_out)
        l2_value2 = l2_value.unsqueeze_(dim=2).unsqueeze_(dim=3)
        return l2_value2

In [30]:

def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)



In [62]:
#Dataset Mnist:
def mnist_data():
    compose = transforms.Compose(
        [transforms.Resize(img_size),
         transforms.ToTensor(),
         transforms.Normalize((0.5,), (0.5,)),  # Remarquez que les paramètres sont des tuples à un élément seulement
         transforms.Grayscale()
         ])
    output_dir = './data/mnist'
    return datasets.MNIST(root=output_dir, train=True,
                          transform=compose, download=True)


mnist = mnist_data()
train_indices = range(0, int(0.8 * len(mnist)))  # Utilisez les 80% premières images pour l'entraînement
train_dataset = Subset(mnist, train_indices)



batch_iterator = DataLoader(train_dataset, shuffle=True, batch_size=batch_size) # List, NCHW format.


cuda = torch.cuda.is_available()
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor



In [59]:

classifier=Classifier()
classifier=classifier.cuda()
classifier.apply(weights_init_normal)
    

Classifier(
  (conv_layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(6, 6), stride=(1, 1), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.18, inplace=True)
  )
  (conv_layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(6, 6), stride=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.18, inplace=True)
  )
  (fc_block1): Sequential(
    (0): Linear(in_features=12800, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.18, inplace=True)
    (3): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [55]:

Image_conv1=classifier.conv_layer1(mnist.data[:16,:,:].unsqueeze(1).cuda().float())
Image_conv2=classifier.conv_layer2(Image_conv1)

[Image_conv1.shape,Image_conv2.shape]

[torch.Size([16, 16, 25, 25]), torch.Size([16, 32, 20, 20])]

In [35]:
classifier(mnist.data[:16,:,:].unsqueeze(1).cuda().float()).shape

torch.Size([16, 10, 1, 1])

In [50]:
display_server="http://localhost"
display_port=8097
try:
    
    vis = visdom.Visdom(server=display_server, port=display_port, raise_exceptions=True) # Create vis env.
except ImportError:
    vis = None
else:
    vis.close(None) # Clear all figures.



Setting up a new session...


In [12]:

for i in range(1):
    for j in range(32):
        vis.image((Image_conv2[1,j,:,:]))
        

In [14]:

for i in range(1):
    for j in range(32):
        vis.image((Image_conv2[1,j,:,:]))
        

In [15]:
for i in range(1):
    print(i)

0


In [37]:
K=random.randint(0,59999)
Image_vis=mnist.data[K,:,:].unsqueeze(0).unsqueeze(1).cuda().float()
Image_vis_conv1=classifier.conv_layer1(Image_vis)
Image_vis_conv2=classifier.conv_layer2(Image_vis_conv1)
for i in range (16):

    vis.image(Image_vis_conv1[0,i,:,:])

for j in range(32):
    vis.image(Image_vis_conv2[0,j,:,:])



In [39]:
lr=0.0001
b1=0.5
b2=0.999
optimizer=optim.Adam(classifier.parameters(),lr=lr,betas=(b1,b2))
loss_C=nn.CrossEntropyLoss()


In [40]:
from tqdm import tqdm

def train(n_epochs):
    for epoch in tqdm(range(n_epochs)):
        print('Epoch {}'.format(epoch))
        for i, (batch, labels) in enumerate(batch_iterator):
            
            imgs = Variable(batch.type(Tensor), requires_grad=False)
            
        
            labels = Variable(labels.type(Tensor), requires_grad=False)
            optimizer.zero_grad()

            output = classifier(imgs)

            loss = loss_C(output, labels.unsqueeze(1).unsqueeze(2).long() )

            loss.backward()
            optimizer.step()


In [41]:
train(1)

  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 0


100%|██████████| 1/1 [03:52<00:00, 232.75s/it]


In [51]:
#K=random.randint(0,59999)   

Image_vis=mnist.data[48000+K,:,:].unsqueeze(0).unsqueeze(1).cuda().float()
Image_vis_conv1=classifier.conv_layer1(Image_vis)
Image_vis_conv2=classifier.conv_layer2(Image_vis_conv1)
vis.image(Image_vis)
for i in range (16):

    vis.image(Image_vis_conv1[0,i,:,:])

for j in range(32):
    vis.image(Image_vis_conv2[0,j,:,:])


In [47]:
test_indices = range(int(0.8 * len(mnist)), len(mnist))
test_dataset = Subset(mnist, test_indices)

mnist.data[test_indices,:,:].unsqueeze(1).cuda().float()

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]],


        ...,


        [[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0.

In [54]:

#K=random.randint(1,12000)
vis.images(mnist.data[48000+K,:,:])
images=mnist.data[test_indices,:,:].unsqueeze(1).cuda().float()
classifier(images[:])[K,:,:,:]



tensor([[[-3.1316]],

        [[-0.7783]],

        [[ 9.0048]],

        [[ 0.5316]],

        [[-4.5609]],

        [[-4.4992]],

        [[-1.5463]],

        [[-2.2132]],

        [[-0.0121]],

        [[-2.4002]]], device='cuda:0', grad_fn=<SliceBackward0>)

In [48]:
labels=mnist.targets[48000:].unsqueeze(1).unsqueeze(2).cuda()   
predicted=torch.argmax(classifier(images[:16]),dim=1)
mean_accuracy=torch.tensor(0)
print(mean_accuracy)
for i in range (750):
       
    predicted=torch.argmax(classifier(images[16*i:16*(i+1)]),dim=1)
    mean_accuracy=torch.sum((torch.sum(labels[16*i:16*(i+1)]==predicted)/16)+mean_accuracy)/2
mean_accuracy*100




tensor(0)


tensor(99.1148, device='cuda:0')